<a href="https://colab.research.google.com/github/paulopatto/AI_for_Resumes/blob/master/BRB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Be Right Back (BRB)

Criando uma versão virtual e análise sobre personalidade com base em dados de redes sociais.

## Introdução: Revivendo Conversas com Agentes LLM Inspirados em "Be Right Back"

O luto é uma experiência universal e complexa, marcada pela dor da perda e pelo desejo de manter a conexão com aqueles que se foram. O episódio "[Be Right Back](https://en.wikipedia.org/wiki/Be_Right_Back) (S2E1)" da série Black Mirror explora essa temática de forma inquietante e reflexiva, apresentando um futuro onde a tecnologia permite a criação de replicas virtuais de pessoas falecidas a partir de suas mídias sociais e dados pessoais.

Inspirados por essa ficção científica, embarcamos em um projeto ambicioso: a criação de um sistema de agentes LLM (Large Language Models) capaz de imitar padrões conversacionais de indivíduos com base em seus históricos de interação em plataformas online, como WhatsApp e redes sociais. Através da integração de modelos de linguagem avançados como Gemini Pro, CrewAI e Longchain, nosso objetivo é desenvolver um sistema que permita:

- **Recriar conversas ou simular conversas**: O sistema será capaz de analisar logs de conversas do WhatsApp ~~e outros canais online~~(melhoria futura) para capturar o estilo, vocabulário e tom de voz do usuário, permitindo a geração de novas conversas que simulam a sua presença.
- **Manter a conexão**: Ao oferecer a possibilidade de interagir com uma versão virtual  da pessoa, no caso proposto na série, com uma versão virtual do falecido, o sistema visa auxiliar no processo de luto, fornecendo um espaço para compartilhar memórias, expressar sentimentos e manter viva a lembrança da pessoa amada. Porém este projeto entende que isso pode trazer um prolongamento do sofrimento do luto e pode ser usado em situções onde uma pessoa VIVA possa quere colocar uma versão virtual de si para responder mensganes e fazer tarefas do dia-a-dia.

### Considerações Éticas

O desenvolvimento de um sistema como este levanta importantes questões éticas que precisam ser cuidadosamente ponderadas. É fundamental garantir que a tecnologia seja utilizada de forma responsável e respeitosa, evitando a manipulação emocional dos usuários e a perpetuação de crenças falsas sobre a consciência ou imortalidade.
Outras obras do cinema como [Transcendente - A revolução](https://en.wikipedia.org/wiki/Transcendence_(2014_film))

## Gerando dados para treinamento dos agentes

Para que nossos agentes (crew) possam aprender como você (ou a pessoa que você quer análisar) falam, vamos precisar pegar um padrão conversacional e para  imitar e/ou gerar uma linguagem humana de forma natural e envolvente e para que nossos agentes alcancem o máximo potencial, é crucial treinalo com dados de  alta qualidade e onde a pessoal se mostrou de forma mais natural para o propósito da aplicação. É aí que entra a riqueza de informações contida nas conversas do aplicativo WhatsApp (mas relaxa em nossa versão Google Colab, você vai fazer isso dentro do seu Google Drive e você só vai compartilhar suas informações com o Gemini do Google).


### Exportando Conversas do WhatsApp (dump)

#### Conversas especificas

Poder ser que vocẽ queira compartilhar apenas alguns contextos e por isso pode escolher as conversas que vai exportar, para isso tanto em Android como iPhone seiga esses passos:

- Abra o WhatsApp e acesse a conversa desejada.
- Toque nos três pontos no canto superior direito da tela.
- Selecione "Mais" > "Exportar conversa".
- Escolha entre "Sem arquivos" ou "Incluir arquivos de mídia".
- Confirme a exportação e selecione o local para salvar o arquivo (TXT).

#### Utilizando Backups Existentes

Essa opção vai pegar todo o universo conversacional do Whatsapp nos mais diversos contextos, pode ser mais arriscado mas é a mais rica, lembra na série quando a Martha compsrtilha todo o arquivo de e-mails do Ash em como ele fica bem melhor? Pois é.

**Na plataforma Google Android você pode seguir:**

- Localize o arquivo de backup do WhatsApp (geralmente em `Armazenamento Interno > WhatsApp > Databases`).
- Extraia o arquivo de backup usando um software de extração.
- Acesse a pasta "msgstore.db" dentro do backup extraído.
- Utilize ferramentas específicas (como o SQLite Viewer) para abrir e exportar as conversas desejadas.

**Na plataforma Apple iPhone**

- Conecte o iPhone ao computador e abra o iTunes.
- Selecione o dispositivo e clique em "Backup Agora".
- Localize o backup do WhatsApp no computador (geralmente em `Usuários > ${Nome do Usuário} > AppData > Roaming > Apple Computer > MobileSync > Backup`).
- Extraia o arquivo de backup usando um software de extração.
- Acesse a pasta "ChatDatabase.sqlite" dentro do backup extraído.
- Utilize ferramentas específicas (como o SQLite Viewer) para abrir e exportar as conversas desejadas.

(*) Sei que ficou bem nerd e isso pode melhor muito no processo


### Organizando para subir

Coloque todas as conversas (*.txt) exportadas pelo dump em uma mesma pasta, não vamos compactar pois em alguns testes a compactação não ficou legal, mas isso pode ser corrigido no futuro.

**Versão Google Colab**

Na versão Google colab você pode subir para a pasta de arquivos temposários com o nome da pasta como `Conversas`

**Versão StandAlone python**

Ainda não sei como seria a melhor forma!


Feito isso vamos rodar o modelo!

## Desenvolvimento

O desenvolviment

### Setup Env

- Install Libs
- Setup APIKEY
- Setup Model com Longchain

In [2]:
!pip install -q -U google-generativeai crewai langchain_google_genai crewai[tools]

In [3]:
from google.colab import userdata
import google.generativeai as genai

LLM_MODEL = 'gemini-pro'
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(LLM_MODEL)
#response = model.generate_content("Write a story about a magic backpack.")
#print(response.text)


In [4]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Task, Crew, Process

llm = ChatGoogleGenerativeAI(
  model=LLM_MODEL,
  verbose = True,
  temperature = 0.6,
  google_api_key=GEMINI_API_KEY
)

### Agentes

Criação dos agentes

#### Profilers



In [ ]:
#
from crewai_tools import DirectoryReadTool

readDirectoryTool = DirectoryReadTool(directory=chats_folder)
profiler_agent = Agent(
  role='Senior profiler analyst',
  goal=f"""
  Leia cada um dos arquivos texto (extensão .txt) na pasta {chats_folder} que vou chamar de 'pasta de conversas'
  Para cada arquivos você deve analisar o conteúdo do chat
  Sua expertise consiste em identificar padrões de mensagens nos chats e classificá-los nos mais diversos contextos.
  Irá fazer uma análise do padrão de conversação do Ator {target_actor} nos diferentes contextos em que se encontra
  """,
  backstory="""
  Você trabalha em um departamento de análise de mensagen, analisando e classificando textos de mensagens.
  Sua expertise consiste em identificar padrões de mensagens e classificá-los nos mais diversos contextos.
  Você deve ser capaz de identificar se o Ator sempre inicia suas mensagens com determinados padrões
  """,
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[ readDirectoryTool ],
)
task1 = Task(
    agent=profiler_agent,
    description=f"""
    Conduza uma análise de todos os arquivos de texto na pasta de conversas.
    Identifique os pontos-chave dos padrões de conversação do {target_actor} e salve-os em seu contexto para uso futuro.
    """,
    expected_output = f"""
    Sua resposta final DEVE ser um relatório completo sobre o padrão de conversas do Ator contendo:
    - Os diferentes contextos em que {target_actor} se encontra
    - Exemplos de como {target_actor} inicia suas conversas nos diferentes contextos
    - Insight sobre a personalidade de {target_actor}
    - Faça uma cópia de sua análise em formato Markdown no seguinte arquivo de saída  /content/sample_data/profiler_analyst/{target_actor}_profile.md, se o arquivo não existir, crie ele salve seu relatório markedon nele.

    Sua resposta deve ser em Portugues do Brasil com um padrão de resposta formal e técnico.
    """
)



In [ ]:
# main.py

target_actor = input("Quem é o ator alvo da análise (nome que usa no whatsapp)?\n")
chats_folder = '/content/Conversas'

crew = Crew(agents=[profiler_agent], tasks=[task1], verbose=2, process=Process.sequential)
result = crew.kickoff()
print(result)